In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.preprocessing import StandardScaler


In [4]:
import sys
import os

sys.path.append(os.path.abspath("../scripts"))

# Now import the modules
from load_data import load_data
from preprocess import preprocess_data
from train_gpr import train_gpr
from evaluate import evaluate_model  # Now should work


In [5]:
# ✅ 2. Load Data
data = load_data()
df_a1 = data["Channel A1.csv"]  # Select dataset

In [7]:
# ✅ 3. Preprocess Data (Drop NaNs, Sample 10%, Normalize)
X_scaled, y_scaled, scaler_X, scaler_y = preprocess_data(df_a1, 'Capacity/mA.h', ['Re(Z)/Ohm', 'Im(Z)/Ohm'])

In [9]:
# ✅ 4. Train Gaussian Process Regressor & Save Model
model_path = "models/gpr_model.pkl"
gp = train_gpr(X_scaled, y_scaled, length_scale=1.0, noise_level=0.1, n_restarts=10)

In [ ]:
# ✅ 5. Generate Test Data for Predictions
X_test = np.linspace(np.min(X_scaled[:, 0]), np.max(X_scaled[:, 0]), 100).reshape(-1, 1)
X_test = np.hstack([X_test, np.full_like(X_test, np.mean(X_scaled[:, 1]))])  # Keep -Im(Z) constant

In [ ]:
# ✅ 6. Make Predictions
y_pred_scaled, sigma = gp.predict(X_test, return_std=True)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()  # Convert back to original scale

In [ ]:
# ✅ 7. Visualize Results
plt.figure(figsize=(8, 5))
plt.scatter(scaler_X.inverse_transform(X_scaled)[:, 0], scaler_y.inverse_transform(y_scaled.reshape(-1, 1)).ravel(), 
            color="red", label="Training Data")
plt.plot(scaler_X.inverse_transform(X_test)[:, 0], y_pred, "b-", label="GP Prediction (mean)")
plt.fill_between(scaler_X.inverse_transform(X_test)[:, 0], 
                 scaler_y.inverse_transform((y_pred_scaled - 2 * sigma).reshape(-1, 1)).ravel(),
                 scaler_y.inverse_transform((y_pred_scaled + 2 * sigma).reshape(-1, 1)).ravel(),
                 color="blue", alpha=0.2, label="Confidence Interval (±2σ)")
plt.xlabel("Re(Z)/Ohm (Real Impedance)")
plt.ylabel("Capacity (mA.h)")
plt.title("Gaussian Process Regression on Battery Impedance Data")
plt.legend()
plt.show()

In [ ]:
# ✅ 8. Evaluate Model Performance on New Test Data
df_test = data["New_Test_Data.csv"]  # Load new test dataset
X_test_scaled, y_test_scaled, _, _ = preprocess_data(df_test, 'Capacity/mA.h', ['Re(Z)/Ohm', '-Im(Z)/Ohm'])

# Load Trained Model & Predict on Test Set
gp = joblib.load(model_path)
y_test_pred_scaled, _ = gp.predict(X_test_scaled, return_std=True)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).ravel()

# Evaluate Model Performance
metrics = evaluate_model(scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).ravel(), y_test_pred)
print("Model Performance:", metrics)
